In [1]:
import pandas as pd
import sqlite3
import csv

In [2]:
conn = sqlite3.connect("S30 ETL Assignment.db")

In [12]:
customers = pd.read_sql_query("select * from customers where age between 18 and 35", conn)

In [13]:
df_customers = pd.DataFrame(customers, columns = ['customer_id','age'])
print(df_customers)

    customer_id  age
0             5   18
1             7   30
2             9   25
3            16   19
4            22   18
5            28   21
6            29   30
7            31   22
8            34   22
9            35   20
10           41   21
11           50   31
12           58   30
13           59   24
14           68   32
15           70   29
16           78   19
17           79   26
18           80   23
19           83   33
20           86   35
21           90   19
22           97   23
23          100   30


In [57]:
orders = pd.read_sql_query("select sales_id, item_id, quantity from orders where quantity is not null;", conn)
df_orders = pd.DataFrame(orders, columns = ['sales_id', 'item_id', 'quantity'])
print(df_orders)

     sales_id  item_id  quantity
0           1        1         2
1           1        3         1
2           2        1         1
3           2        2         1
4           2        3         2
..        ...      ...       ...
942       499        1         2
943       499        3         2
944       500        1         1
945       500        2         2
946       500        3         4

[947 rows x 3 columns]


In [58]:
items = pd.read_sql_query("select item_id from items", conn)
df_items = pd.DataFrame(items, columns=['item_id'])
print(df_items)

   item_id
0        1
1        2
2        3


In [59]:
sales = pd.read_sql_query("select * from sales", conn)
df_sales = pd.DataFrame(sales, columns = ['sales_id','customer_id'])
print(df_sales)

     sales_id  customer_id
0           1           10
1           2           85
2           3           42
3           4           93
4           5           75
..        ...          ...
495       496           22
496       497           68
497       498           77
498       499           10
499       500           31

[500 rows x 2 columns]


In [60]:
inner_cust_sales = pd.merge(df_customers, df_sales, on='customer_id', how='inner')

In [61]:
inner_cust_sales

,customer_id,age,sales_id
0,5,18,48
1,5,18,171
2,5,18,183
3,5,18,405
4,7,30,29
...,...,...,...
123,97,23,147
124,97,23,463
125,100,30,254
126,100,30,322


In [62]:
inner_sales_order = pd.merge(df_sales, df_orders, on='sales_id', how = 'inner')
inner_sales_order

,sales_id,customer_id,item_id,quantity
0,1,10,1,2
1,1,10,3,1
2,2,85,1,1
3,2,85,2,1
4,2,85,3,2
...,...,...,...,...
942,499,10,1,2
943,499,10,3,2
944,500,31,1,1
945,500,31,2,2


In [63]:
inner_items_orders = pd.merge(df_items, df_orders, on='item_id', how = 'inner')
inner_items_orders

,item_id,sales_id,quantity
0,1,1,2
1,1,2,1
2,1,3,3
3,1,4,2
4,1,5,1
...,...,...,...
942,3,496,1
943,3,497,1
944,3,498,1
945,3,499,2


In [64]:
result1 = pd.merge(inner_cust_sales, inner_sales_order, on = ['customer_id', 'sales_id'], how = 'inner')

In [65]:
result1

,customer_id,age,sales_id,item_id,quantity
0,5,18,171,1,1
1,5,18,171,2,3
2,5,18,171,3,1
3,5,18,183,1,1
4,5,18,183,3,2
...,...,...,...,...,...
233,100,30,254,1,1
234,100,30,254,2,1
235,100,30,254,3,2
236,100,30,322,3,1


In [67]:
result2 = pd.merge(result1, inner_items_orders, on = ['sales_id','quantity','item_id'], how = 'inner')

In [68]:
result2

,customer_id,age,sales_id,item_id,quantity
0,5,18,171,1,1
1,5,18,171,2,3
2,5,18,171,3,1
3,5,18,183,1,1
4,5,18,183,3,2
...,...,...,...,...,...
233,100,30,254,1,1
234,100,30,254,2,1
235,100,30,254,3,2
236,100,30,322,3,1


In [74]:
final_df=result2.groupby(['customer_id', 'age','item_id']).agg({'quantity':'sum'}).reset_index()

In [77]:
final_df.to_csv('python_csv',index = False,sep=';', mode = 'w')